In [28]:
import os;
import json;
import builtins
import pandas as pd;

dir = '/home/chesetti/Repos/KVector_Merge/sponge'
testcase = 'join_books_8'
test_dir = os.path.join(dir, testcase)
results_dir = os.path.join(test_dir, 'outputs', 'results')
csv_dir = os.path.join(test_dir, 'csv')
os.makedirs(csv_dir, exist_ok=True)

runs = [os.path.join(results_dir, run) for run in os.listdir(results_dir)]
print(runs)

['/home/chesetti/Repos/KVector_Merge/sponge/merge_lognormal_8/outputs/results/run']


In [29]:
# Walk all the json files and put them in a dataframe

test_results = []
for run in runs:
    for test_result_file in os.listdir(run):
        json_file = builtins.open(os.path.join(run, test_result_file))
        test_result = json.load(json_file)
        test_result['run'] = run
        test_results.append(test_result)
        json_file.close()
test_dataframe = pd.json_normalize(test_results)
print(test_dataframe.columns)

        

Index(['command', 'run', 'result.checksum', 'result.duration_ns',
       'result.duration_sec', 'result.inner_disk_fetch',
       'result.inner_disk_fetch_size', 'result.inner_total_bytes_fetched',
       'result.outer_disk_fetch', 'result.outer_disk_fetch_size',
       'result.outer_total_bytes_fetched', 'spec.algo', 'spec.algo_name',
       'spec.common_key', 'spec.index.leaf_size_in_pages', 'spec.index.search',
       'spec.index.type', 'spec.inner_table', 'spec.key_size', 'spec.key_type',
       'spec.load_sstable_in_mem', 'spec.name', 'spec.num_threads',
       'spec.outer_table', 'spec.result_path', 'spec.value_size',
       'spec.write_result_to_disk'],
      dtype='object')


# Reports

### Overall Duration

In [30]:
overall_duration = test_dataframe.pivot_table(index='spec.common_key', columns='spec.algo_name', values='result.duration_ns', aggfunc='median')
overall_duration.to_csv(os.path.join(csv_dir, 'duration_sec.csv'))
print(overall_duration.columns)
print(overall_duration.index)
print(overall_duration)
throughput = overall_duration
'''
for column in throughput.columns:
    throughput[column] = (200_000_000/throughput.index) * (1000_000_000.0 / throughput[column])
    throughput[column +"_sj-rel"] = (throughput[column] - throughput["sj"] / throughput["sj"]) * 100.0
print(throughput)
overall_duration[["btree2048", "btree1024", "btree256", "sj"]].plot() 
overall_duration[["pgm128", "pgm512", "pgm1024", "sj"]].plot() 
'''

Index(['btree1024', 'btree2048', 'btree256', 'pgm1024', 'pgm2048', 'pgm256',
       'standard_merge'],
      dtype='object', name='spec.algo_name')
Int64Index([10, 20, 30, 40, 50, 60, 70, 80, 90, 100], dtype='int64', name='spec.common_key')
spec.algo_name    btree1024   btree2048    btree256     pgm1024     pgm2048  \
spec.common_key                                                               
10               2150067029  2185985051  2169116812  2121462013  2151303531   
20               2046907549  2040639002  2067968591  2027088874  2034082340   
30               1993928094  2022156592  2041891828  2009876658  2028422468   
40               1965946730  1948634812  1985065876  1950886641  1950735440   
50               1994644546  2012543086  2000789825  1967830211  1984854432   
60               1963545049  1942207121  1962511127  1973910396  1931466651   
70               2003318268  1952639341  1972995793  1981577570  1967813132   
80               1927669763  1941566412  1959918

'\nfor column in throughput.columns:\n    throughput[column] = (200_000_000/throughput.index) * (1000_000_000.0 / throughput[column])\n    throughput[column +"_sj-rel"] = (throughput[column] - throughput["sj"] / throughput["sj"]) * 100.0\nprint(throughput)\noverall_duration[["btree2048", "btree1024", "btree256", "sj"]].plot() \noverall_duration[["pgm128", "pgm512", "pgm1024", "sj"]].plot() \n'

In [31]:
def parseIndexName(name):
    index = name
    epsilon = 0
    if name=="btree1024":
        index="BTree"
        epsilon=1024
    if name=="btree2048":
        index="BTree"
        epsilon=2048
    if name=="btree256":
        index="BTree"
        epsilon=256
    if name=="pgm1024":
        index="PGM"
        epsilon=2049
    if name=="pgm512":
        index="PGM"
        epsilon=1025
    if name=="pgm128":
        index="PGM"
        epsilon=257
    return {"name": index, "epsilon": epsilon}


inner_index_size = test_dataframe.pivot_table(index='spec.common_key', columns='spec.algo_name', values='result.inner_index_size', aggfunc='median')
"select c1,sum(c2), group by c1 from table"
test_dataframe.filter([""]).group_by(c1, )

data_by_epsilon = []
data = {}
data['indexes'] = []
data['memory_bytes'] = []
for index in inner_index_size:
    data['indexes'].append(index)
    data['memory_bytes'].append(inner_index_size[index].mean())
    index_dict = parseIndexName(index)
    index_dict['memory'] = inner_index_size[index].mean()
    data_by_epsilon.append(index_dict)
inner_index_size = pd.DataFrame(data=data)
inner_index_size.to_csv(os.path.join(csv_dir, 'inner_index_size.csv'))
print(inner_index_size)
inner_index_size_by_eps = pd.DataFrame(data_by_epsilon).pivot(index='epsilon', columns='name',values='memory')
inner_index_size_by_eps.to_csv(os.path.join(csv_dir, 'inner_index_size_by_eps.csv'))
print(inner_index_size_by_eps)

KeyError: 'result.inner_index_size'

In [ ]:
inner_index_build_duration = test_dataframe.pivot_table(index='spec.common_key', columns='spec.algo_name', values='result.inner_index_build_duration_ns', aggfunc='median')
data_by_epsilon = []
data = {}
data['indexes'] = []
data['build_duration'] = []
for index in inner_index_build_duration:
    data['indexes'].append(index)
    data['build_duration'].append(inner_index_build_duration[index].mean())
    index_dict = parseIndexName(index)
    index_dict['build_duration'] = inner_index_build_duration[index].mean()
    data_by_epsilon.append(index_dict)
inner_index_build_duration = pd.DataFrame(data=data)
inner_index_build_duration.to_csv(os.path.join(csv_dir, 'inner_index_build_duration_ns.csv'))
print(inner_index_build_duration)
inner_index_bd_by_eps = pd.DataFrame(data_by_epsilon).pivot(index='epsilon', columns='name',values='build_duration')
inner_index_bd_by_eps.to_csv(os.path.join(csv_dir, 'inner_index_size_by_eps.csv'))
print(inner_index_bd_by_eps)

KeyError: 'result.inner_index_build_duration_ns'

In [ ]:
inner_index_fetch = test_dataframe.pivot_table(index='spec.common_key', columns='spec.algo_name', values='result.inner_disk_fetch', aggfunc='median')
inner_index_total_fetch = test_dataframe.pivot_table(index='spec.common_key', columns='spec.algo_name', values='result.inner_total_bytes_fetched', aggfunc='median')
inner_index_fetch_size = test_dataframe.pivot_table(index='spec.common_key', columns='spec.algo_name', values='result.inner_disk_fetch_size', aggfunc='median')
print(inner_index_fetch)
print(inner_index_total_fetch)
print(inner_index_fetch_size)

spec.algo_name   btree1024  btree2048  btree256  pgm1024  pgm2048  pgm256  \
spec.common_key                                                             
10                  195313      97657    781250   156246    86806  390456   
50                  195313      97657    776614   156076    86786  384787   
100                 195310      97657    721113   155420    86707  367419   

spec.algo_name       sj  
spec.common_key          
10               781250  
50               781250  
100              781250  
spec.algo_name    btree1024   btree2048    btree256     pgm1024     pgm2048  \
spec.common_key                                                               
10               3200000000  3200000000  3200000000  3199904704  3199980560   
50               3200000000  3200000000  3181010944  3196429648  3199252352   
100              3199950848  3200000000  2953678848  3182988416  3196353008   

spec.algo_name       pgm256          sj  
spec.common_key                          
10  

In [ ]:
result_checksum = (test_dataframe[['spec.common_key', 'spec.algo_name', 'result.checksum']].sort_values(by=['spec.common_key', 'spec.algo_name'])) #.loc[test_dataframe['spec.common_key']=='10'])
for common_key in sorted(test_dataframe['spec.common_key'].unique()):
    checksums = result_checksum.loc[result_checksum['spec.common_key'] == common_key]
    unique_checksums = checksums['result.checksum'].unique()
    if (len(unique_checksums) == 1):
        print(f"common_key: {common_key} checksum: {unique_checksums} OK")
    else:
        print(f"common_key: {common_key} checksums don't match")
print(result_checksum.loc[result_checksum['spec.common_key'] == 100])
print(result_checksum.loc[result_checksum['spec.common_key'] == 100]['result.checksum'].unique())

common_key: 10 checksum: ['9B57CDDD9798604CC7D3BC254D867BE9'] OK
common_key: 50 checksum: ['BF415AFBB5A73E61F504EDC6FCCA9870'] OK
common_key: 100 checksum: ['5B625421C7CCBDC2FF99DDCC00517618'] OK
    spec.common_key spec.algo_name                   result.checksum
1               100      btree1024  5B625421C7CCBDC2FF99DDCC00517618
24              100      btree1024  5B625421C7CCBDC2FF99DDCC00517618
34              100      btree1024  5B625421C7CCBDC2FF99DDCC00517618
3               100      btree2048  5B625421C7CCBDC2FF99DDCC00517618
42              100      btree2048  5B625421C7CCBDC2FF99DDCC00517618
55              100      btree2048  5B625421C7CCBDC2FF99DDCC00517618
6               100       btree256  5B625421C7CCBDC2FF99DDCC00517618
8               100       btree256  5B625421C7CCBDC2FF99DDCC00517618
61              100       btree256  5B625421C7CCBDC2FF99DDCC00517618
10              100        pgm1024  5B625421C7CCBDC2FF99DDCC00517618
20              100        pgm1024  5B625421C